In [1]:
!pip install xgboost pandas scikit-learn matplotlib


   ---------------------------------------- 0.0/72.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/72.0 MB ? eta -:--:--
   ---------------------------------------- 0.8/72.0 MB 6.7 MB/s eta 0:00:11
   -- ------------------------------------- 4.2/72.0 MB 14.0 MB/s eta 0:00:05
   ---- ----------------------------------- 8.1/72.0 MB 15.7 MB/s eta 0:00:05
   ------ --------------------------------- 12.1/72.0 MB 18.0 MB/s eta 0:00:04
   ---------- ----------------------------- 18.6/72.0 MB 19.6 MB/s eta 0:00:03
   ------------- -------------------------- 23.6/72.0 MB 20.2 MB/s eta 0:00:03
   ---------------- ----------------------- 28.8/72.0 MB 21.0 MB/s eta 0:00:03
   ------------------ --------------------- 34.1/72.0 MB 21.6 MB/s eta 0:00:02
   --------------------- ------------------ 39.3/72.0 MB 21.9 MB/s eta 0:00:02
   ------------------------ --------------- 44.6/72.0 MB 22.2 MB/s eta 0:00:02
   -------------------------- ------------- 48.5/72.0 MB 22.2 MB/s eta 0:0

In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import ndcg_score

import matplotlib.pyplot as plt

print("✅ Imports successful")
print("XGBoost version:", xgb.__version__)


✅ Imports successful
XGBoost version: 3.1.2


In [3]:
np.random.seed(42)

num_jobs = 50
msp_per_job = 10
total_rows = num_jobs * msp_per_job


In [4]:
data = {
    "job_id": np.repeat(np.arange(num_jobs), msp_per_job),
    "msp_id": np.tile(np.arange(msp_per_job), num_jobs),
    "distance_km": np.random.uniform(1, 30, total_rows),
    "price_quote": np.random.uniform(500, 2000, total_rows),
    "past_accept_rate": np.random.uniform(0.3, 0.9, total_rows),
    "completion_rate": np.random.uniform(0.7, 1.0, total_rows),
    "rating": np.random.uniform(3.0, 5.0, total_rows),
}


In [6]:
df = pd.DataFrame(data)



In [1]:
df["relevance"] = (
    0.3 * (1 / df["distance_km"]) +
    0.3 * (df["past_accept_rate"]) +
    0.3 * (df["completion_rate"]) +
    0.1 * (df["rating"] / 5)
)


NameError: name 'df' is not defined